# Análise e Visualização da População NEET (15-24 anos)

**Autor:** Gemini & Ligia

**Fonte de Dados:** `people_14_25_2023_2024_fullvars.csv`

**Objetivo:** Este notebook analisa os microdados da PNAD Contínua para o 4º trimestre de 2024, calcula a população NEET total e por sexo, e gera um gráfico para visualizar os resultados.

### 1. Importar Bibliotecas

Primeiro, importamos as bibliotecas necessárias para a análise: `pandas` para manipulação de dados e `matplotlib` para a criação de gráficos.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os

print("Bibliotecas importadas com sucesso!")

### 2. Carregar os Dados

Agora, vamos carregar o nosso ficheiro CSV pré-processado para um DataFrame do pandas. O script verifica se o ficheiro existe antes de tentar carregá-lo.

In [ ]:
input_filename = 'people_14_25_2023_2024_fullvars.csv'

if not os.path.exists(input_filename):
    print(f"\nErro: O ficheiro de entrada '{input_filename}' não foi encontrado.")
    print("Por favor, certifique-se de que o ficheiro está na mesma pasta que este notebook.")
else:
    try:
        print(f"A carregar dados de '{input_filename}'...")
        df = pd.read_csv(input_filename)
        print("Dados carregados com sucesso!")
        # Exibir as primeiras linhas para verificação
        display(df.head())
    except Exception as e:
        print(f"\nErro: Não foi possível ler o ficheiro '{input_filename}'. Pode estar corrompido. Erro: {e}")

### 3. Filtrar Dados para Análise

Vamos focar a nossa análise apenas nos dados relevantes:
1.  **Período:** 4º trimestre de 2024.
2.  **Idade:** Jovens entre 15 e 24 anos.

In [ ]:
print("A filtrar dados para o 4º Trimestre de 2024 e idade entre 15-24 anos...")

# Filtrar pelo ano e trimestre específicos
df_periodo = df[(df['Year'] == 2024) & (df['Quarter'] == 4)].copy()

# Filtrar pelo grupo de idade alvo
df_idade = df_periodo[(df_periodo['Age'] >= 15) & (df_periodo['Age'] <= 24)].copy()

if df_idade.empty:
    print("\nErro: Nenhum dado encontrado para jovens de 15 a 24 anos no 4º Trimestre de 2024.")
else:
    print(f"Filtro aplicado. Encontradas {len(df_idade)} observações de jovens no período.")

### 4. Identificar População NEET e Calcular Totais

Nesta etapa, aplicamos a definição de NEET para filtrar os indivíduos que não estudam e não trabalham. Em seguida, usamos os pesos amostrais (`Weight_V1028`) para estimar a população real e imprimimos os resultados.

In [ ]:
print("A identificar a população NEET e a calcular os totais...")

# Identificar indivíduos NEET
df_neet = df_idade[
    (df_idade['School_label'] == 'No') &
    (df_idade['Occupation_label'] != 'Occupied')
].copy()

# Calcular totais usando os pesos amostrais ('Weight_V1028')
total_jovens = df_idade['Weight_V1028'].sum()
total_neet = df_neet['Weight_V1028'].sum()
homens_neet = df_neet[df_neet['Sex_label'] == 'Male']['Weight_V1028'].sum()
mulheres_neet = df_neet[df_neet['Sex_label'] == 'Female']['Weight_V1028'].sum()

# Calcular percentagem
percentagem_neet = (total_neet / total_jovens) * 100 if total_jovens > 0 else 0

print("\n--- Resultados para o 4º Trimestre de 2024 (15-24 anos) ---")
print(f"População Total de Jovens: {total_jovens:,.0f}")
print(f"Total de Jovens NEET: {total_neet:,.0f} ({percentagem_neet:.1f}%)")
print(f"  - Homens NEET: {homens_neet:,.0f}")
print(f"  - Mulheres NEET: {mulheres_neet:,.0f}")

### 5. Gerar Gráfico

Finalmente, criamos um gráfico de barras para visualizar a distribuição da população NEET, incluindo o total e a divisão por sexo.

In [ ]:
print("\nA gerar o gráfico...")

# Dados para o gráfico
categorias = ['Total NEET', 'Homens', 'Mulheres']
valores = [total_neet, homens_neet, mulheres_neet]
cores = ['#003f5c', '#58508d', '#bc5090']

fig, ax = plt.subplots(figsize=(10, 7))

# Criar as barras
bars = ax.bar(categorias, valores, color=cores)

# Adicionar título e rótulos
ax.set_title('Análise da População NEET (15-24 anos) - 4º Trimestre de 2024', fontsize=16, pad=20)
ax.set_ylabel('População Estimada', fontsize=12)
ax.tick_params(axis='x', labelsize=12)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.yaxis.grid(True, linestyle='--', alpha=0.6)
ax.get_yaxis().set_major_formatter(plt.FuncFormatter(lambda x, loc: "{:,.0f}".format(int(x))))

# Adicionar rótulos de dados no topo das barras
for bar in bars:
    yval = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2.0, yval + (max(valores) * 0.01), f'{yval:,.0f}', ha='center', va='bottom', fontsize=11)

# Adicionar texto de resumo abaixo do título
summary_text = (
    f"População Total de Jovens (15-24 anos) no período: {total_jovens:,.0f}\n"
    f"Jovens NEET representam {percentagem_neet:.1f}% do total"
)
fig.text(0.5, 0.9, summary_text, ha='center', fontsize=12, style='italic', color='gray')

plt.tight_layout(rect=[0, 0, 1, 0.9]) # Ajustar layout para dar espaço ao texto
plt.show()